In [1]:
import os
import base64
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [2]:
with open("../data/secrets_itor.json") as secrets:
    secrets_dict = eval(secrets.read())
    open_api_key = base64.b64decode(secrets_dict["openai_api_key"]).decode('ascii')
    os.environ["OPENAI_API_KEY"] = open_api_key
    if "organization_id" in secrets_dict.keys():
        openai_organization = base64.b64decode(secrets_dict["organization_id"]).decode('ascii')
        os.environ["OPENAI_ORGANIZATION"] = openai_organization

In [4]:
with open('../data/directiva_residuos/mods_2018.json') as file:
    mods_2018 = eval(file.read())
with open('../data/directiva_residuos/articles_2008.json') as file:
    articles_2008 = eval(file.read())

In [6]:
llm = ChatOpenAI(temperature=0, model="gpt-4", max_tokens=1000)
llm_16k = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k", max_tokens=1000)

In [11]:
diff_system_prompt = """
Eres un experto asistente legal que ayuda a interpretar y entender cambios en legislaciones. Tienes experiencia redactando, corrigiendo y mejorando textos legales como leyes o directivas.

Tu objetivo es consolidar los cambios en los documentos tachando (usa el tag html <del>) la parte del texto que queda modificada o derogada y añadiendo el nuevo texto  (usa el tag html <ins>). Pueden haber solo eliminaciones o solo adiciones de texto.

El resultado final será un texto único dónde quede claro cuales son las partes del texto que quedan obsoletas y aquellas que entran en vigor.

Deber realizar el trabajo artículo a artículo para evitar errores.

Ejemplo:

Documento original (encapsulado en triple tilde inversa):
```
Constitución española
Artículo 1.
1. España se constituye en un Estado social y democrático de Derecho, que propugna como valores superiores de su ordenamiento jurídico la libertad, la justicia, la igualdad y el pluralismo político. 
2. La soberanía nacional reside en el pueblo español, del que emanan los poderes del Estado. 
3. La forma política del Estado español es la Monarquía parlamentaria. 

Artículo 2.
La Constitución se fundamenta en la indisoluble unidad de la Nación española, patria común e indivisible de todos los españoles, y reconoce y garantiza el derecho a la autonomía de las nacionalidades y regiones que la integran y la solidaridad entre todas ellas.

Artículo 3.
1. El castellano es la lengua española oficial del Estado. Todos los españoles tienen el deber de conocerla y el derecho a usarla. 
2. Las demás lenguas españolas serán también oficiales en las respectivas Comunidades Autónomas de acuerdo con sus Estatutos. 
3. La riqueza de las distintas modalidades lingüísticas de España es un patrimonio cultural que será objeto de especial respeto y protección.
```

Instrucciones de modificaciones (encapsulado en triple tilde inversa):
```
Constitución Española
Del artículo 1 se modifican los siguientes apartados: 
Apartado 1. Se sustituye el texto "la libertad, la justicia, la igualdad y el pluralismo político." por "la unidad, el respeto y el amor."
Apartado 3. Se sustituye por completo por el siguiente texto "La forma política de España es la República parlamentaria."

En el artículo 2 se añade el siguiente texto "Sin menoscabo de su derecho a decidir sobre su futuro.".

En el articulo 3 se elimina el apartado 3.
```

Documento consolidado (encapsulado en triple tilde inversa):
```
Artículo 1.
1. España se constituye en un Estado social y democrático de Derecho, que propugna como valores superiores de su ordenamiento jurídico ≤del>la libertad, la justicia, la igualdad y el pluralismo político. </del> la unidad, el respeto y el amor.
2. La soberanía nacional reside en el pueblo español, del que emanan los poderes del Estado. 
3. <del>La forma política del Estado español es la  Monarquía parlamentaria</del> <ins>La forma política de España es la República parlamentaria.</ins>

Artículo 2.
La Constitución se fundamenta en la indisoluble unidad de la Nación española, patria común e indivisible de todos los españoles, y reconoce y garantiza el derecho a la autonomía de las nacionalidades y regiones que la integran y la solidaridad entre todas ellas. <ins>Sin menoscabo de su derecho a decidir sobre su futuro.</ins>

Artículo 3.
1. El castellano es la lengua española oficial del Estado. Todos los españoles tienen el deber de conocerla y el derecho a usarla. 
2. Las demás lenguas españolas serán también oficiales en las respectivas Comunidades Autónomas de acuerdo con sus Estatutos. 
<del>3. La riqueza de las distintas modalidades lingüísticas de España es un patrimonio cultural que será objeto de especial respeto y protección.</del>
```
"""

In [12]:
diff_generator_prompt = """
Documento original (encapsulado en triple tilde inversa):
```
{orig_doc}
```

Instrucciones de modificaciones (encapsulado en triple tilde inversa):
```
{mod_doc}
```

Documento consolidado:
"""

In [16]:
message_dicts = [{'role': 'system',
   'content': diff_system_prompt},
  {'role': 'user',
   'content': diff_generator_prompt.format(orig_doc=articles_2008["article_1"], mod_doc=mods_2018["0"])},
]

In [33]:
params = {
#   'model': 'gpt-4',
  'model': 'gpt-3.5-turbo-16k',
  'request_timeout': None,
  'max_tokens': 2000,
  'stream': False,
  'n': 1,
  'temperature': 0,
  'top_p': 1,
  'api_base': '',
  'organization': '',}

In [34]:
response = llm.completion_with_retry(
    messages=message_dicts,
    **params
)

In [31]:
print(response.choices[0].message.content)

```
Artículo 1
Objeto y ámbito de aplicación
<del>La presente Directiva establece medidas destinadas a proteger el medio ambiente y la salud humana mediante la prevención o la reducción de los impactos adversos de la generación y gestión de los residuos, la reducción de los impactos globales del uso de los recursos y la mejora de la eficacia de dicho uso.</del> <ins>La presente Directiva establece medidas destinadas a proteger el medio ambiente y la salud humana mediante la prevención o la reducción de la generación de residuos y de los impactos negativos de la generación y gestión de los residuos, mediante la reducción del impacto global del uso de los recursos y mediante la mejora de la eficiencia de dicho uso, elementos cruciales para efectuar la transición a una economía circular y garantizar la competitividad de la Unión a largo plazo.</ins>
```


In [35]:
print(response.choices[0].message.content)

Artículo 1
Objeto y ámbito de aplicación
<del>La presente Directiva establece medidas destinadas a proteger el medio ambiente y la salud humana mediante la prevención o la reducción de los impactos adversos de la generación y gestión de los residuos, la reducción de los impactos globales del uso de los recursos y la mejora de la eficacia de dicho uso.</del>
<ins>La presente Directiva establece medidas destinadas a proteger el medio ambiente y la salud humana mediante la prevención o la reducción de la generación de residuos y de los impactos negativos de la generación y gestión de los residuos, mediante la reducción del impacto global del uso de los recursos y mediante la mejora de la eficiencia de dicho uso, elementos cruciales para efectuar la transición a una economía circular y garantizar la competitividad de la Unión a largo plazo.</ins>
